In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Read the train and test data set first

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train= pd.read_csv('../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv')
test= pd.read_csv('../input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv')
train.head()

In [ ]:
train.shape

Will look for the data description 

In [ ]:
train.info()

In [ ]:
#print the unique values in the object variables
print('Married: ' + str(train['Married'].unique()))
print('Dependents: '+ str(train['Dependents'].unique()))
print('Education: '+ str(train['Education'].unique()))
print('Self_Employed: '+ str(train['Self_Employed'].unique()))
print('Property_Area: '+ str(train['Property_Area'].unique()))

In [ ]:
print('Loan status',train['Loan_Status'].value_counts())



The data set is not balanced in outputs

In [ ]:
train.describe()

# Checking the missing values in the dataset

In [ ]:
# will look for the missing vlaues in the data
train.isnull().sum().sort_values(ascending=False)

In [ ]:
#checking the relatioship between the variables and target 
import seaborn as sns

sns.countplot(train['Loan_Status'],hue=train['Credit_History'])

In [ ]:
print(pd.crosstab(train['Credit_History'],train['Loan_Status']))

In [ ]:
train['Credit_History'].value_counts()

In [ ]:
#will impute the missing values by 1 since 1 have the most frequesnt values in the dataset
train['Credit_History']= train['Credit_History'].fillna(1.0)

In [ ]:
sns.countplot(train['Loan_Status'],hue=train['Credit_History'])

In [ ]:
#next we have self employed column to look 
sns.countplot(train['Loan_Status'],hue=train['Self_Employed'])

In [ ]:
train.Self_Employed.value_counts()

In [ ]:

#fill the missing values by No in data set
train['Self_Employed']= train['Self_Employed'].fillna('No')

In [ ]:
sns.countplot(train['Loan_Status'],hue=train['Self_Employed'])

Next we have Loan amount variable to look and since its a numerical variable we will look for scatter plot

In [ ]:
plt.scatter(train['Loan_Status'],train['LoanAmount'])

In [ ]:
#will fill the missing values with the mean value
train['LoanAmount']=train['LoanAmount'].fillna(train.LoanAmount.mean())

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
train['Dependents'].isnull().sum()/len(train['Dependents'])

In [ ]:
#drop the other missing values from the data
train.dropna(inplace=True)

In [ ]:
train.describe()

#  Handling the outliers from the dataset 

In [ ]:
plt.boxplot(train['ApplicantIncome'])

In [ ]:
plt.boxplot(train['CoapplicantIncome'])

In [ ]:
#creating a new variable called total income by adding applicant income + coapplicant income

train['TotalIncome']= train['ApplicantIncome']+train['CoapplicantIncome']

In [ ]:
sns.boxplot(x='Education',y='ApplicantIncome',data=train)

We can see that graduate people also having same outliers since they are graduate their income must be higher we will chekck the same with coapplicant income

In [ ]:
sns.distplot(train.TotalIncome,kde=False)

In [ ]:

train = train.drop(columns=['Loan_ID'],axis=1)

In [ ]:
#Next, make all other columns numerical as well. 
train['Married'] = np.where((train['Married'] == 'Yes'), 1, 0)
train['Gender'] = np.where((train['Gender'] == 'Female'), 1, 0)
train['Education'] = np.where((train['Education'] == 'Graduate'), 1, 0)
train['Self_Employed'] = np.where((train['Self_Employed'] == 'Yes'), 1, 0)
train['Dependents'] = np.where((train['Dependents'] == '0'), 0, 1)
train['Loan_Status'] = np.where((train['Loan_Status'] == 'Y'), 1, 0)

In [ ]:
train['Property_Area'].value_counts()

In [ ]:
#Doing label encoding the categorical features
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Property_Area']=le.fit_transform(train['Property_Area'])

In [ ]:
train.head()

In [ ]:
y= train['Loan_Status']
X= train.drop(columns=['Loan_Status'])

Selecting Important features 

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Name of the column','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))

In [ ]:
corrmat = train.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(10,10))

#plot heat map
g=sns.heatmap(train[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Married,credit history,coapplicantincome,loan amount,total income having high importance in variables list

In [ ]:
X= X[['Married','Credit_History','TotalIncome','CoapplicantIncome','LoanAmount','ApplicantIncome']]

In [ ]:
X.head()

In [ ]:
X.shape,y.shape

In [ ]:
y.value_counts()

dataset is not balanced so we use smote algorithm to balance the output

In [ ]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X, y = sm.fit_sample(X, y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
#spliiitng the test train
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=6)

In [ ]:
x_train.shape,x_test.shape

In [ ]:
#fitting Logistic regression
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()

In [ ]:
log.fit(x_train,y_train)

In [ ]:
import pickle


In [ ]:
#saving my model to pickle file
pickle.dump(log,open('model_pkl','wb'))

In [ ]:
model = pickle.load(open('model_pkl','rb'))

In [ ]:
x_test

In [ ]:
q= model.predict([[1,1.0,459.000000,0.000000,25.000000,3459]])

In [ ]:
q[0]

In [ ]:
log.score(x_train,y_train)

In [ ]:
#Predicting test dataset
pred=log.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

In [ ]:
from sklearn import metrics
metrics.confusion_matrix(y_test,pred)

In [ ]:
print(classification_report(y_test,pred))

Will try with Decision tree

In [ ]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_train,y_train)

In [ ]:
pred1=clf.predict(x_test)

In [ ]:
metrics.confusion_matrix(y_test,pred1)

In [ ]:
print(classification_report(y_test,pred1))

Logistic regression performed well out of two models